In [1]:
from IPython.display import display
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import Float, String, Integer, DateTime
from geoalchemy2 import Geometry
import requests
import json

# Database configuration
POSTGRES_USER = "user"
POSTGRES_PASSWORD = "password"
POSTGRES_DB = "urban_data"
POSTGRES_HOST = "postgres"
POSTGRES_PORT = "5432"

# Create a connection string
connection_string = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
engine = create_engine(connection_string)

# Load datasets
link_info_path = '/home/jovyan/work/data/link_info.parquet.gz'
speed_data_path = '/home/jovyan/work/data/duval_jan1_2024.parquet.gz'
link_info_url = 'https://cdn.urbansdk.com/data-engineering-interview/link_info.parquet.gz'
speed_data_url = 'https://cdn.urbansdk.com/data-engineering-interview/duval_jan1_2024.parquet.gz'

# Download dataset func
def download_file(url, local_path):
    response = requests.get(url)
    response.raise_for_status() 
    with open(local_path, 'wb') as file:
        file.write(response.content)

# Fazer o download dos arquivos
download_file(link_info_url, link_info_path)
download_file(speed_data_url, speed_data_path)

link_info_df = pd.read_parquet(link_info_path)
speed_data_df = pd.read_parquet(speed_data_path)

In [2]:
display(link_info_df)

,link_id,_length,road_name,usdk_speed_category,funclass_id,speedcat,volume_value,volume_bin_id,volume_year,volumes_bin_description,geo_json
0,1148855686,0.027340324,None,40,4,2,800,1,2022,0-1999,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
1,1240632857,0.019262501,E 21st St,40,4,5,5000,3,2022,5000-7449,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
2,1240632858,0.021126614,E 21st St,40,4,5,5000,3,2022,5000-7449,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
3,1240474884,0.037903631,University Blvd W,40,4,5,18000,6,2022,15000-19999,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
4,1240959781,0.022369356,None,40,4,5,3500,2,2022,2000-4999,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
...,...,...,...,...,...,...,...,...,...,...,...
100919,1296092422,0.007456452,Arlington Rd N,40,4,5,14000,5,2022,10000-14999,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
100920,1296555008,0.021747985,Edgewood Ave N,40,4,5,6000,3,2022,5000-7449,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
100921,1295361824,0.024233469,Collins Rd,40,4,5,9000,4,2022,7500-9999,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
100922,1295361813,0.006835081,Collins Rd,40,4,5,9500,4,2022,7500-9999,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."


In [3]:
# Define functions to transform data for insertion
def transform_link_info(df):
    # Ensure geo_json is a valid geometry type for PostGIS
    df['geometry'] = df['geo_json'].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)
    return df.drop(columns=['geo_json'])  # Drop the geo_json column after transformation

def transform_speed_data(df):
    # Ensure date_time is in a DateTime format
    df['date_time'] = pd.to_datetime(df['date_time'])
    return df

# Transform the datasets
link_info_df = transform_link_info(link_info_df)
speed_data_df = transform_speed_data(speed_data_df)

In [4]:
display(link_info_df)

,link_id,_length,road_name,usdk_speed_category,funclass_id,speedcat,volume_value,volume_bin_id,volume_year,volumes_bin_description,geometry
0,1148855686,0.027340324,None,40,4,2,800,1,2022,0-1999,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
1,1240632857,0.019262501,E 21st St,40,4,5,5000,3,2022,5000-7449,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
2,1240632858,0.021126614,E 21st St,40,4,5,5000,3,2022,5000-7449,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
3,1240474884,0.037903631,University Blvd W,40,4,5,18000,6,2022,15000-19999,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
4,1240959781,0.022369356,None,40,4,5,3500,2,2022,2000-4999,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
...,...,...,...,...,...,...,...,...,...,...,...
100919,1296092422,0.007456452,Arlington Rd N,40,4,5,14000,5,2022,10000-14999,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
100920,1296555008,0.021747985,Edgewood Ave N,40,4,5,6000,3,2022,5000-7449,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
100921,1295361824,0.024233469,Collins Rd,40,4,5,9000,4,2022,7500-9999,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."
100922,1295361813,0.006835081,Collins Rd,40,4,5,9500,4,2022,7500-9999,"{""type"":""MultiLineString"",""coordinates"":[[[-81..."


In [7]:
# Insert data into the existing tables
def insert_data(df, table_name):
    df.to_sql(table_name, engine, if_exists="append", index=False)

# Insert data with transformations applied
insert_data(link_info_df, "link_info")
insert_data(speed_data_df, "speed_data")

print("Data inserted successfully.")

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "link_info_pkey"
DETAIL:  Key (link_id)=(1148855686) already exists.

[SQL: INSERT INTO link_info (link_id, _length, road_name, usdk_speed_category, funclass_id, speedcat, volume_value, volume_bin_id, volume_year, volumes_bin_description, geometry) VALUES (%(link_id__0)s, %(_length__0)s, %(road_name__0)s, %(usdk_speed_catego ... 249618 characters truncated ... , %(volume_bin_id__999)s, %(volume_year__999)s, %(volumes_bin_description__999)s, %(geometry__999)s)]
[parameters: {'speedcat__0': 2, 'volume_year__0': 2022, 'volume_value__0': 800, 'geometry__0': '{"type":"MultiLineString","coordinates":[[[-81.51023,30.16599],[-81.51038,30.16637]]]}', '_length__0': Decimal('0.027340324'), 'funclass_id__0': 4, 'volume_bin_id__0': 1, 'road_name__0': None, 'usdk_speed_category__0': 40, 'link_id__0': 1148855686, 'volumes_bin_description__0': '0-1999', 'speedcat__1': 5, 'volume_year__1': 2022, 'volume_val

In [ ]:
# Apply Coordinates System to DB
from sqlalchemy import text

query = """
UPDATE link_info
SET geometry = ST_SetSRID(geometry, 4326)
WHERE ST_SRID(geometry) IS DISTINCT FROM 4326;
"""

with engine.connect() as connection:
    connection.execute(text(query))
    connection.commit() 

In [ ]:
# Load data into DataFrames
link_info_df = pd.read_sql_table("link_info", engine)
speed_data_df = pd.read_sql_table("speed_data", engine)

# Check the first few rows and data types
print("Link Info Data:")
display(link_info_df.head())
print("Speed Data:")
display(speed_data_df.head())

# Summary statistics
print("Summary Statistics - Link Info Data:")
display(link_info_df.describe())
print("Summary Statistics - Speed Data:")
display(speed_data_df.describe())

In [ ]:
# Check for missing values
print("Missing values in Link Info Data:")
print(link_info_df.isnull().sum())
print("Missing values in Speed Data:")
print(speed_data_df.isnull().sum())


In [ ]:
# Query to calculate average speed by link_id, period, and day of the week
query = """
SELECT
    s.link_id,
    AVG(s.average_speed) AS avg_speed,
    l.road_name,
    l._length AS length,
    l.geometry AS geo_json,
    s.period
FROM
    speed_data s
JOIN
    link_info l ON s.link_id = l.link_id
GROUP BY
    s.link_id, s.period, s.day_of_week, l.road_name, l._length, l.geometry
"""

# Fetch data into a DataFrame
transformed_df = pd.read_sql_query(query, engine)
transformed_df

In [ ]:
# Query to find  the names of the roads in a given bounding box
query = """
SELECT DISTINCT
    road_name,
    ST_AsText(geometry) AS geometry_text
FROM 
    link_info
WHERE 
    geometry && ST_MakeEnvelope(-81.7, 30.2, -81.5, 30.4, 4326)
AND
    road_name <> 'None';
"""
# Fetch data into a DataFrame
roads_df = pd.read_sql_query(query, engine)
display(roads_df)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Average speed by day of the week
plt.figure(figsize=(10, 6))
sns.boxplot(data=transformed_df, x="period", y="avg_speed")
plt.title("Distribution of Average Speeds by Time Period")
plt.xlabel("Period")
plt.ylabel("Average Speed")
plt.show()

# Average speed by link and period
avg_speed_by_link = transformed_df.groupby("link_id")["avg_speed"].mean()
plt.figure(figsize=(12, 6))
avg_speed_by_link.plot(kind="hist", bins=30)
plt.title("Distribution of Average Speeds across Links")
plt.xlabel("Average Speed")
plt.ylabel("Frequency")
plt.show()


In [ ]:
from shapely import wkb
import json
from shapely.geometry import shape

# Convert GeoJSON strings to Shapely geometries
transformed_df['geometry'] = transformed_df['geo_json'].apply(lambda x: shape(json.loads(x)) if x else None)

# Extract coordinates from Shapely geometries
# For MultiLineString geometries, we'll use the centroid for simplicity
transformed_df['coordinates'] = transformed_df['geometry'].apply(lambda geom: geom.centroid.coords[0] if geom else (None, None))

# Separate longitude and latitude
transformed_df['longitude'] = transformed_df['coordinates'].apply(lambda x: x[0])
transformed_df['latitude'] = transformed_df['coordinates'].apply(lambda x: x[1])

# Display the resulting DataFrame
transformed_df[['link_id', 'avg_speed', 'longitude', 'latitude']].head()

# Scatter plot of locations colored by average speed
plt.figure(figsize=(10, 8))
sns.scatterplot(data=transformed_df, x="longitude", y="latitude", hue="avg_speed", palette="coolwarm", size="avg_speed")
plt.title("Spatial Distribution of Links by Average Speed")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


In [ ]:
# Select numeric columns for correlation analysis
corr_features = transformed_df[['avg_speed', 'length', 'longitude', 'latitude']]
corr_matrix = corr_features.corr()

# Display correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Correlation Matrix")
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Select features for modeling
X = transformed_df[['length', 'longitude', 'latitude', 'period']]
y = transformed_df['avg_speed']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Model evaluation
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R^2 Score:", r2_score(y_test, y_pred))


In [ ]:
# Average speed by period
avg_speed_by_period = transformed_df.groupby('period')['avg_speed'].mean()
print(avg_speed_by_period)

In [ ]:
# Convert DataFrame to GeoJSON
geojson_data = {
    "type": "FeatureCollection",
    "features": []
}

for _, row in transformed_df.iterrows():
    # Convert geometry if it's stored as a string
    geometry = json.loads(row["geo_json"]) if isinstance(row["geo_json"], str) else row["geo_json"]

    feature = {
        "type": "Feature",
        "geometry": geometry,
        "properties": {
            "link_id": row["link_id"],
            "avg_speed": row["avg_speed"],
            "road_name": row["road_name"],
            "length": row["length"]
        }
    }
    geojson_data["features"].append(feature)


In [ ]:
# Save GeoJSON data to a file for verification
with open("test_geojson.json", "w") as f:
    json.dump(geojson_data, f, indent=2)


In [ ]:
from mapboxgl.utils import create_color_stops
from mapboxgl.viz import ChoroplethViz
from IPython.display import IFrame
import json

# Mapbox token
mapbox_token = "pk.eyJ1IjoiZ3VpYm9ydG9sYXNvIiwiYSI6ImNtMndoajR4czA2d2sybXB0enNhc2dtcjAifQ.PHvelUdPZ4tW7WQL0fvNmQ"

# Define color stops for visualization
color_stops = create_color_stops([20, 40, 60, 80], colors=['blue', 'green', 'yellow', 'red'])

# Create the map visualization
viz = ChoroplethViz(
    geojson_data,
    access_token=mapbox_token,
    color_property="avg_speed",
    color_stops=color_stops,
    center=(-81.6556, 30.3322), 
    zoom=10,
    line_stroke="blue"
)

# Save the map visualization to HTML
viz.create_html('map_visualization.html')

# Display the HTML file in an IFrame
IFrame('map_visualization.html', width=700, height=500)


# 1. ST_Distance: Calculate the distance between two geometries
Purpose: Measures the shortest distance between two geometries. This is useful for proximity analysis, such as finding nearby points of interest.

In [ ]:

query = """
SELECT 
    a.link_id AS link_a,
    b.link_id AS link_b,
    ST_Distance(a.geometry::geography, b.geometry::geography) AS distance_meters
FROM 
    link_info a, 
    link_info b
WHERE 
    a.link_id < b.link_id
ORDER BY 
    distance_meters
LIMIT 10;
"""

distance_df = pd.read_sql_query(query, engine)
distance_df.head()

# 2. ST_Intersects: Check if two geometries intersect
Purpose: Determines if two geometries share any space. This is useful for filtering geometries that overlap or touch.

In [ ]:
query = """
SELECT 
    a.link_id AS link_a,
    b.link_id AS link_b
FROM 
    link_info a, 
    link_info b
WHERE 
    a.link_id < b.link_id
    AND ST_Intersects(a.geometry, b.geometry);
"""

intersect_df = pd.read_sql_query(query, engine)
intersect_df.head()

# 3. ST_Within: Check if one geometry is within another
Purpose: Returns true if one geometry is entirely within another. This is useful for containment analysis, such as finding all features within a defined area.

In [ ]:
query = """
SELECT 
    link_id,
    road_name
FROM 
    link_info
WHERE 
    ST_Within(geometry, ST_MakeEnvelope(-81.7, 30.2, -81.5, 30.4, 4326));
"""

geo_within_df = pd.read_sql_query(query, engine)

# 4. ST_Length: Calculate the length of a line geometry
Purpose: Calculates the length of a line geometry, useful for road or path length calculations.

In [ ]:
query = """
SELECT 
    link_id,
    road_name,
    ST_Length(geometry::geography) AS length_meters
FROM 
    link_info
ORDER BY 
    length_meters DESC
LIMIT 10;
"""

lenght_df = pd.read_sql_query(query, engine)
display(lenght_df)

# 5. ST_Transform: Change the spatial reference system (SRID) of a geometry
Purpose: Transforms a geometry to a new SRID, useful for converting between coordinate systems.

In [ ]:
query = """
SELECT 
    link_id,
    road_name,
    ST_Transform(geometry, 3857) AS geom_web_mercator
FROM 
    link_info
WHERE 
    ST_Intersects(geometry, ST_MakeEnvelope(-81.7, 30.2, -81.5, 30.4, 4326));
"""

change_ref_df = read_sql_query(query, engine)
display(change_ref_df)